#**Realignment of Regional Office for RAM Wireless**

### Install Required Libraries
This step installs the necessary Python libraries for the project:
- `amplpy`: Interface for running AMPL models in Python.
- `ampltools`: For managing AMPL licenses and tools.
- `pandas`: For data manipulation and preprocessing.
- `openpyxl`: For handling Excel files.


In [ ]:
# Import necessary libraries
!pip install -q amplpy ampltools pandas openpyxl

### Initialize AMPL Environment
This block sets up the AMPL environment for use within Google Colab or a local machine:
- **Modules**: Specifies solvers such as `coin`, `gurobi`, `cplex`, etc.
- **AMPL Object**: Instantiates the AMPL object, using either a local installation or the notebook environment.
- **Register Magics**: Allows the use of `%ampl_eval` and `%ampl_set` commands in the notebook.


In [ ]:

# Google Colab & AMPL integration
MODULES, LICENSE_UUID = ["coin", 'gurobi', "cplex", "highs", "gokestrel"], "42fc7eb6-69aa-445d-b655-3ad24d836541"
from amplpy import tools
from ampltools import cloud_platform_name, ampl_notebook, register_magics

# instantiate AMPL object and register magics
if cloud_platform_name() is None:
    ampl = AMPL() # Use local installation of AMPL
else:
    ampl = tools.ampl_notebook(modules=MODULES, license_uuid=LICENSE_UUID, g=globals())

register_magics(ampl_object=ampl)

Licensed to Bundle #6741.7193 expiring 20241231: INFO 645 Prescriptive Analytics, Prof. Paul Brooks, Virginia Commonwealth University.


### Import Necessary Libraries
This block imports all required libraries for the project:
- `pandas`: For data manipulation and preprocessing.
- `numpy`: For handling numerical operations.
- `google.colab.files`: To handle file uploads in Google Colab.


In [ ]:
# Import Necessary Libraries
import pandas as pd  # Data manipulation and preprocessing
import numpy as np  # Numerical operations
from google.colab import files  # File uploads in Google Colab


### Access Data from Google Drive
Use this code block if you're working in Google Colab and want to load the dataset from your Google Drive.  
**Important**:  
Ensure that the dataset has been cleaned to remove any metadata (e.g., extra headers or notes).  
Uncleaned metadata can interfere with Python and AMPL processing, causing errors.



In [ ]:
# Mount Google Drive to access the dataset
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Load Dataset from Google Drive
This code block provides the option to load the dataset from Google Drive. However, we recommend caution when using this method due to potential issues:

- **Metadata Issue**: The original dataset contained metadata (e.g., extra headers and notes) that caused significant issues for Python and AMPL processing. To avoid this, ensure you are using the **cleaned version of the dataset**.
- **File Path Adjustment**: Update the `file_path` to match the exact location of your dataset in Google Drive.

**Note**:  
If you encounter issues with loading the data using this method, refer to the next code block to upload the dataset directly from your machine. This approach is simpler and more reliable.


In [ ]:


# Encountered multiple issues with loading the dataset from Google Drive.
# If this method doesn't work for you, use the next code block to upload the file directly from your machine for a simpler and more reliable approach (Make sure to use our cleaned dataset as the metadata caused issues for Python).


#Group file path
# # Specify the file path (adjust to where you saved the cleaned dataset)
file_path = '/content/drive/MyDrive/645 - PRESCRIPTIVE ANALYSIS/SIMULATION PROBLEMS/realignment_data_forAMPL.xlsx'

# # Load sheets and clean data (convert "--" to NaN for missing values)
store_data = pd.read_excel(file_path, sheet_name='Store Data').replace("--", np.nan)
regional_office_data = pd.read_excel(file_path, sheet_name='Regional Office Data').replace("--", np.nan)
travel_data = pd.read_excel(file_path, sheet_name='Travel').replace("--", np.nan)


<ipython-input-5-ed366694c603>:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  travel_data = pd.read_excel(file_path, sheet_name='Travel').replace("--", np.nan)


### Upload Dataset Directly from Your Machine (Recommended)
This code block allows you to upload the dataset directly from your local machine, avoiding the need for Google Drive.

**Important**:  
- Use the cleaned version of the dataset: `'realignment_data_forAMPL.xlsx'`.  
- The cleaned dataset ensures that issues caused by metadata (e.g., extra headers or notes) in the original file are avoided.  
- This method is simpler and more reliable compared to the Google Drive option.


In [ ]:


# Upload the file
#uploaded = files.upload()

# Assumes the uploaded filename is 'realignment_data_forAMPL.xlsx' (Cleaned version with metadata removed etc.)
#file_path = list(uploaded.keys())[0]  # Get the uploaded file's name

# Load sheets and convert "--" to NaN for missing values
#store_data = pd.read_excel(file_path, sheet_name='Store Data').replace("--", np.nan)
#regional_office_data = pd.read_excel(file_path, sheet_name='Regional Office Data').replace("--", np.nan)
#travel_data = pd.read_excel(file_path, sheet_name='Travel').replace("--", np.nan)


### Define Sets and Prepare Data Dictionaries for Optimization
This code block prepares the necessary data dictionaries for the optimization problem, including adjustments for round trips.

**Key Adjustments and Details**:
1. **Round Trip Adjustments**:
   - Both travel times and distances are doubled to account for round trips.
2. **Handling Missing Data**:
   - Missing values (`NaN`) in the travel dataset are excluded from the dictionaries, as they represent locations that do not travel to certain regional offices.
3. **Dictionaries for Optimization**:
   - Data is converted into dictionaries that map relevant relationships (e.g., distance between offices and stores, hours required per store per area).
4. **Set Definitions**:
   - Defines sets for **Regional Offices (R)**, **Stores (S)**, and **Service Areas (A)**.

**Important Note**:
- The cleaned version of the dataset (`realignment_data_forAMPL.xlsx`) was required to ensure proper processing, as the original metadata caused issues.


In [ ]:
# Define sets based on data columns
R = regional_office_data['Unnamed: 0'].tolist()  # Regional offices
S = store_data['Unnamed: 0'].tolist()  # Stores
A = ['Inventory', 'Payroll', 'Hiring', 'Marketing', 'Merchandising']  # Areas

# Prepare distance dictionary, handling missing values by excluding NaNs
distance = travel_data.melt(id_vars='Unnamed: 0',
                            value_vars=['Staunton Mileage', 'Warrenton Mileage', 'Richmond Mileage', 'Tappahannock Mileage'],
                            var_name='Office', value_name='Distance')
distance['Office'] = distance['Office'].str.replace(' Mileage', '')
# Create dictionary, excluding entries with NaN values and doubling distances for round trips
distance_dict = {(row['Office'], row['Unnamed: 0']): row['Distance'] * 2  # Double for round trips
                 for _, row in distance.iterrows() if pd.notna(row['Distance'])}

# Prepare travel_time dictionary, handling missing values by excluding specific (Office, Store) pairs
travel_time = travel_data.melt(id_vars='Unnamed: 0',
                               value_vars=['Staunton Travel Time', 'Warrenton Travel Time', 'Richmond Travel Time', 'Tappahannock Travel Time'],
                               var_name='Office', value_name='Travel Time')
travel_time['Office'] = travel_time['Office'].str.replace(' Travel Time', '')
# Create dictionary, excluding specific (Office, Store) pairs with NaN values
travel_time_dict = {(row['Office'], row['Unnamed: 0']): row['Travel Time'] * 2  # Double travel times for round trips
                    for _, row in travel_time.iterrows() if pd.notna(row['Travel Time'])}

# Prepare hours_avail dictionary with correct keys (office, area)
hours_avail = regional_office_data.melt(id_vars='Unnamed: 0',
                                        value_vars=['Inventory Hours Available', 'Payroll Hours Available',
                                                    'Hiring Hours Available', 'Marketing Hours Available',
                                                    'Merchandising Hours Available'],
                                        var_name='Area', value_name='Hours Available')
hours_avail['Area'] = hours_avail['Area'].str.replace(' Hours Available', '')  # Remove extra text
hours_avail_dict = {(row['Unnamed: 0'], row['Area']): row['Hours Available'] for _, row in hours_avail.iterrows()}

# Prepare hours_req dictionary with correct keys (store, area)
hours_req = store_data.melt(id_vars='Unnamed: 0',
                            value_vars=['Inventory Hours', 'Payroll Hours', 'Hiring Hours',
                                        'Marketing Hours', 'Merchandising Hours'],
                            var_name='Area', value_name='Hours Required')
hours_req['Area'] = hours_req['Area'].str.replace(' Hours', '')  # Remove extra text
hours_req_dict = {(row['Unnamed: 0'], row['Area']): row['Hours Required'] for _, row in hours_req.iterrows()}

# Prepare trips dictionary with correct keys (store, area)
trips = store_data.melt(id_vars='Unnamed: 0',
                        value_vars=['Inventory Trips', 'Payroll Trips', 'Hiring Trips',
                                    'Marketing Trips', 'Merchandising Trips'],
                        var_name='Area', value_name='Trips Required')
trips['Area'] = trips['Area'].str.replace(' Trips', '')  # Remove extra text
trips_dict = {(row['Unnamed: 0'], row['Area']): row['Trips Required'] for _, row in trips.iterrows()}


### Validate and Clean Data Dictionaries
This code block ensures the data dictionaries used in the optimization are clean and free of errors.

**Details and Adjustments**:
1. **Validation and Cleaning**:
   - A custom function (`clean_and_validate_dict`) is used to:
     - Replace any remaining `NaN` values with `0`.
     - Print a warning if any non-numeric value is detected, skipping those entries.
2. **Targeted Cleaning**:
   - Only specific dictionaries (`hours_avail_dict`, `hours_req_dict`, and `trips_dict`) are cleaned, as `distance_dict` and `travel_time_dict` were already cleaned earlier.
3. **Ensuring Numeric Values**:
   - Validates that all dictionary values are numeric (either integers or floats) to prevent errors during optimization.

**Key Note**:
- This step is necessary to ensure the integrity of data passed into the optimization model.


In [ ]:
# Function to validate dictionary and replace NaNs (only applied where needed)
def clean_and_validate_dict(data_dict, dict_name):
    cleaned_dict = {}
    for key, value in data_dict.items():
        # Ensure value is numeric, replace NaN with 0
        if pd.isna(value):
            value = 0
        elif not isinstance(value, (int, float)):
            print(f"Non-numeric value found in {dict_name} at {key}: {value}")
            continue
        cleaned_dict[key] = value
    return cleaned_dict

# No need to clean distance_dict and travel_time_dict, as NaNs have already been handled during their creation
# Clean and validate only the necessary dictionaries
hours_avail_dict = clean_and_validate_dict(hours_avail_dict, "hours_avail")
hours_req_dict = clean_and_validate_dict(hours_req_dict, "hours_req")
trips_dict = clean_and_validate_dict(trips_dict, "trips")


### AMPL Optimization Model Implementation
This code block defines and solves the optimization model using AMPL. The model calculates the optimal assignment of stores to regional offices to minimize travel costs while satisfying capacity constraints.

**Key Adjustments and Workarounds**:
1. **Round Trip Adjustments**:
   - Both `travel_time` and `distance` are multiplied by 2 to account for round trips. This adjustment was necessary after discovering that previous calculations underestimated the total cost by only considering one-way trips.

2. **Handling Missing Travel Data (`"--"` / NaN Values)**:
   - Missing data in the `travel` sheet was initially represented as `"--"`, which was converted to `NaN` during data preprocessing.
   - For the AMPL model, a **prohibitively large number** (`LARGE = 1e6`) is used as a workaround. This ensures:
     - Missing (office, store) combinations are effectively excluded from optimization due to their unrealistically high cost.
   - This workaround resolves the issue of Python and AMPL misinterpreting missing values during the calculation.

3. **Parameters and Decision Variables**:
   - The cost parameters (`mileage_rate` and `hourly_rate`) are used to calculate the objective function.
   - Binary decision variable `x` is used to determine store-to-office assignments.

4. **Objective Function**:
   - Minimizes the total travel cost, which includes mileage and travel time costs for all store-regional office assignments.

5. **Constraints**:
   - **Assignment Constraint**: Each store must be assigned to exactly one regional office.
   - **Capacity Constraint**: Total required hours (work + travel) for each regional office must not exceed the available employee hours.

**Important Notes**:
- The cleaned version of the dataset was critical to avoid metadata issues during processing.
- The debug option (`display _conname, _con.sstatus, _con.slack`) can help identify any constraint violations or slack variables after solving the model.


In [ ]:
# Set cost parameters
mileage_rate = 0.585
hourly_rate = 26

# Define the AMPL model
ampl.eval('''
reset;
set R; # Regional offices
set S; # Stores
set A; # Service areas

# Parameters
param mileage_rate;
param hourly_rate;
param LARGE; # Large value to represent prohibitively high costs
param distance {i in R, j in S} default LARGE; # Large value for round trips between regional offices and stores if missing
param travel_time {i in R, j in S} default LARGE; # Large value for round-trip travel time between offices and stores if missing
param hours_avail {i in R, k in A}; # Available hours per area in each regional office
param hours_req {j in S, k in A}; # Hours required per area in each store
param trips {j in S, k in A}; # Annual trips required per area per store

# Decision Variables
var x {i in R, j in S} binary; # 1 if store j is assigned to regional office i, 0 otherwise

# Objective: Minimize total travel costs (mileage and salary)
minimize total_travel_cost:
    sum {i in R, j in S, k in A} trips[j, k] * (mileage_rate * distance[i, j] + hourly_rate * travel_time[i, j]) * x[i, j];

# Constraints
# 1. Assignment Constraint: Each store must be assigned to only one regional office
subject to assignment_constraint {j in S}:
    sum {i in R} x[i, j] = 1;

# 2. Capacity Constraints: Total required hours (work + travel) for each area in each office must not exceed available hours
subject to capacity_constraint {i in R, k in A}:
    sum {j in S} (hours_req[j, k] + travel_time[i, j] * trips[j, k]) * x[i, j] <= hours_avail[i, k];
''')

# Set data in AMPL
ampl.set['R'] = R
ampl.set['S'] = S
ampl.set['A'] = A

ampl.param['mileage_rate'] = mileage_rate
ampl.param['hourly_rate'] = hourly_rate
ampl.param['LARGE'] = 1e6  # Set a large value for prohibitive cost
ampl.param['distance'] = distance_dict
ampl.param['travel_time'] = travel_time_dict
ampl.param['hours_avail'] = hours_avail_dict
ampl.param['hours_req'] = hours_req_dict
ampl.param['trips'] = trips_dict

# Solve the model
ampl.solve()

# Debugging: Display constraint slacks after solving
# ampl.eval('display _conname, _con.sstatus, _con.slack;')


Error executing "solve" command:
No solver specified:  option solver is ''.


In [ ]:
# Specify the solver and solve the model
ampl.setOption('solver', 'cplex')
ampl.solve()

CPLEX 22.1.1: CPLEX 22.1.1: optimal solution; objective 195479.31
17 simplex iterations


### Solve the Optimization Model and Display Results
This code block solves the optimization model using the specified solver (`CPLEX`) and prints the total travel cost in a clear and professional format.

**Key Updates**:
- The total cost is presented in a user-friendly message, suitable for sharing with higher-level stakeholders.
- The cost accounts for all constraints, round trips, and adjusted parameters.


In [ ]:
# Specify the solver and solve the model
ampl.setOption('solver', 'cplex')
ampl.solve()

# Display the total travel cost in a user-friendly message
total_cost = ampl.getObjective('total_travel_cost').value()
print(f"The optimal total travel cost for store-to-regional office assignments, accounting for round trips and capacity constraints, is: ${total_cost:,.2f}.")


CPLEX 22.1.1: CPLEX 22.1.1: optimal solution; objective 195479.31
17 simplex iterations
The optimal total travel cost for store-to-regional office assignments, accounting for round trips and capacity constraints, is: $195,479.31.


### Display Store-to-Regional Office Assignments
This code block processes the results of the optimization and displays the optimal assignments of stores (counties) to regional offices in a clear and user-friendly format.

**Steps**:
1. Extracts the results of the binary decision variable `x` (assignments) directly from the model as a Python dictionary.
2. Filters the assignments to include only `(Office, Store)` pairs where `x[i, j] == 1`, indicating an assignment.
3. Iterates through the filtered results to present each store's assignment to its respective regional office in a concise format.

**Why This is Useful**:
- Converts raw optimization results into a clear, professional summary for stakeholders.
- Avoids reliance on AMPL's display command, ensuring results are handled entirely within Python for flexibility.


In [ ]:
# Extract assignments directly from x_result
x_result = ampl.getVariable('x').getValues().toDict()

# Filter and process only the assignments where x[i, j] == 1
assignments = [(store, office) for (office, store), value in x_result.items() if value == 1]

# Print the assignments in a user-friendly format
print("\nOptimal Store-to-Regional Office Assignments:")
for store, office in assignments:
    print(f"{store} -> {office}")



Optimal Store-to-Regional Office Assignments:
Charles_City_County -> Richmond
Chesterfield_County -> Richmond
City_of_Richmond -> Richmond
Cumberland_County -> Richmond
Dinwiddie_County -> Richmond
Fluvanna_County -> Richmond
Goochland_County -> Richmond
Hanover_County -> Richmond
Henrico_County -> Richmond
Hopewell_County -> Richmond
James_City_County -> Richmond
Louisa_County -> Richmond
New_Kent_County -> Richmond
Powhatan_County -> Richmond
Prince_George_County -> Richmond
Albemarle_County -> Staunton
Amherst_County -> Staunton
Augusta_County -> Staunton
Buckingham_County -> Staunton
Greene_County -> Staunton
Madison_County -> Staunton
Nelson_County -> Staunton
Rockbridge_County -> Staunton
Rockingham_County -> Staunton
Caroline_County -> Tappahannock
City_of_Fredericksburg -> Tappahannock
Essex_County -> Tappahannock
King_George_County -> Tappahannock
King_William_County -> Tappahannock
King_and_Queen_County -> Tappahannock
Mathews_County -> Tappahannock
Spotsylvania_County -> Ta

# **PART B**

In [ ]:
# Display the assignments
ampl.display('x')

x [*,*] (tr)
:                      Richmond Staunton Tappahannock Warrenton    :=
Albemarle_County           0        1          0           0
Amherst_County             0        1          0           0
Augusta_County             0        1          0           0
Buckingham_County          0        1          0           0
Caroline_County            0        0          1           0
Charles_City_County        1        0          0           0
Chesterfield_County        1        0          0           0
City_of_Fredericksburg     0        0          1           0
City_of_Richmond           1        0          0           0
Culpeper_County            0        0          0           1
Cumberland_County          1        0          0           0
Dinwiddie_County           1        0          0           0
Essex_County               0        0          1           0
Fauquier_County            0        0          0           1
Fluvanna_County            1        0          0           0
Go

In [ ]:
#ampl.eval('''expand;''')
#ampl.eval('''expand x;''')

In [ ]:
# Display constraint slacks for debugging
print("\n--- Constraint Slacks ---")
ampl.eval('display _conname, _con.sstatus, _con.slack;')

# Check for infeasibility or unbounded solutions
solution_status = ampl.getValue("solve_result")
if solution_status in ['infeasible', 'unbounded']:
    print(f"Warning: Solution is {solution_status}. Please review the model and input data.")
else:
    print(f"Solution status: {solution_status}")



--- Constraint Slacks ---
# $2 = _con.sstatus
:                          _conname                           $2  _con.slack
 :=
1    "assignment_constraint['Albemarle_County']"             none      0
2    "assignment_constraint['Amherst_County']"               none      0
3    "assignment_constraint['Augusta_County']"               none      0
4    "assignment_constraint['Buckingham_County']"            none      0
5    "assignment_constraint['Caroline_County']"              none      0
6    "assignment_constraint['Charles_City_County']"          none      0
7    "assignment_constraint['Chesterfield_County']"          none      0
8    "assignment_constraint['City_of_Fredericksburg']"       none      0
9    "assignment_constraint['City_of_Richmond']"             none      0
10   "assignment_constraint['Culpeper_County']"              none      0
11   "assignment_constraint['Cumberland_County']"            none      0
12   "assignment_constraint['Dinwiddie_County']"             none    

**Observations**

Geographic Misalignment:

1) Stafford_County is assigned to Richmond, even though Warrenton is geographically closer.

2) King_William_County, King_and_Queen_County, and York_County are assigned to Tappahannock, which might align with cost efficiency but could disrupt operations due to travel times.

These assignments indicate that the optimization model prioritized cost over geographic feasibility.

# **PART C**

**Remedies and Ramifications**

1) Introduce Geographic Constraints:

Add a constraint to limit the maximum allowable distance or travel time between a store and its assigned regional office:

In [ ]:
# Define a maximum distance or travel time threshold
MAX_DISTANCE = 100
MAX_TRAVEL_TIME = 2

ampl.eval(f'''
subject to geographic_distance_constraint {{i in R, j in S}}:
    distance[i, j] <= {MAX_DISTANCE};

subject to geographic_time_constraint {{i in R, j in S}}:
    travel_time[i, j] <= {MAX_TRAVEL_TIME};
''')

# Solve the model with geographic constraints
ampl.solve()

# Extract and display the updated solution
total_cost_with_constraints = ampl.getObjective('total_travel_cost').value()
print(f"Total Cost with Geographic Constraints: ${total_cost_with_constraints:,.2f}")

x_result_constrained = ampl.getVariable('x').getValues().toPandas()
print("\nUpdated Assignments with Geographic Constraints:")
print(x_result_constrained[x_result_constrained['x.val'] > 0.5])


	presolve, constraint geographic_distance_constraint['Richmond','Albemarle_County']:
		no variables, but lower bound = -Infinity, upper = -40
	presolve, constraint geographic_distance_constraint['Richmond','Amherst_County']:
		no variables, but lower bound = -Infinity, upper = -140
	presolve, constraint geographic_distance_constraint['Richmond','Augusta_County']:
		no variables, but lower bound = -Infinity, upper = -999900
	presolve, constraint geographic_distance_constraint['Richmond','Buckingham_County']:
		no variables, but lower bound = -Infinity, upper = -44
	presolve, constraint geographic_distance_constraint['Richmond','City_of_Fredericksburg']:
		no variables, but lower bound = -Infinity, upper = -18
	523 presolve messages suppressed.
Total Cost with Geographic Constraints: $195,479.31

Updated Assignments with Geographic Constraints:
                                     x.val
index0       index1                       
Richmond     Charles_City_County         1
             Che

2) Manually Adjust Assignments:

Reassign stores like Stafford_County to their nearest office (Warrenton) based on geographic intuition:

In [ ]:
ampl.eval('''
subject to reassign_stafford:
    x['Warrenton', 'Stafford_County'] = 1;
''')

# Solve the model with manual adjustments
ampl.solve()

# Extract and display the updated solution
total_cost_manual_adjustments = ampl.getObjective('total_travel_cost').value()
print(f"Total Cost with Manual Adjustments: ${total_cost_manual_adjustments:,.2f}")

x_result_manual = ampl.getVariable('x').getValues().toPandas()
print("\nUpdated Assignments with Manual Adjustments:")
print(x_result_manual[x_result_manual['x.val'] > 0.5])


	presolve, constraint geographic_distance_constraint['Richmond','Albemarle_County']:
		no variables, but lower bound = -Infinity, upper = -40
	presolve, constraint geographic_distance_constraint['Richmond','Amherst_County']:
		no variables, but lower bound = -Infinity, upper = -140
	presolve, constraint geographic_distance_constraint['Richmond','Augusta_County']:
		no variables, but lower bound = -Infinity, upper = -999900
	presolve, constraint geographic_distance_constraint['Richmond','Buckingham_County']:
		no variables, but lower bound = -Infinity, upper = -44
	presolve, constraint geographic_distance_constraint['Richmond','City_of_Fredericksburg']:
		no variables, but lower bound = -Infinity, upper = -18
	523 presolve messages suppressed.
Total Cost with Manual Adjustments: $194,130.69

Updated Assignments with Manual Adjustments:
                                     x.val
index0       index1                       
Richmond     Charles_City_County         1
             Chesterfiel

**Compare the Results of All Remedies**

In [ ]:
import pandas as pd

# Process the original results (already correct)
x_result = x_result[['Regional Office', 'Store', 'Assignment']]  # This is already structured correctly

# Process Remedy 1 results
x_result_constrained = x_result_constrained.reset_index()  # Reset the multi-index
x_result_constrained.columns = ['Regional Office', 'Store', 'Assignment']  # Rename columns

# Process Remedy 2 results
x_result_manual = x_result_manual.reset_index()  # Reset the multi-index
x_result_manual.columns = ['Regional Office', 'Store', 'Assignment']  # Rename columns

# Combine the results for comparison
comparison_df = pd.concat([
    x_result.set_index(['Regional Office', 'Store'])['Assignment'],  # Original
    x_result_constrained.set_index(['Regional Office', 'Store'])['Assignment'],  # Remedy 1
    x_result_manual.set_index(['Regional Office', 'Store'])['Assignment']  # Remedy 2
], axis=1)
comparison_df.columns = ['Original', 'With Constraints', 'Manual Adjustments']

# Display assignments that differ across remedies
print("\n--- Assignment Comparison Across Remedies ---")
differences = comparison_df[(comparison_df['Original'] != comparison_df['With Constraints']) |
                             (comparison_df['Original'] != comparison_df['Manual Adjustments']) |
                             (comparison_df['With Constraints'] != comparison_df['Manual Adjustments'])]
print(differences)



--- Assignment Comparison Across Remedies ---
                                     Original  With Constraints  \
Regional Office Store                                             
Warrenton       Stafford_County           1.0                 0   
Richmond        Albemarle_County          NaN                 0   
                Amherst_County            NaN                 0   
                Augusta_County            NaN                 0   
                Buckingham_County         NaN                 0   
...                                       ...               ...   
Warrenton       Rockbridge_County         NaN                 0   
                Rockingham_County         NaN                 0   
                Spotsylvania_County       NaN                 0   
                Westmoreland_County       NaN                 0   
                York_County               NaN                 0   

                                     Manual Adjustments  
Regional Office Store  

In [ ]:
# Print summary of total costs for Original, Remedy 1, and Remedy 2
print("\n--- Comparison of Remedies ---")
print(f"Original Total Cost: ${total_cost:,.2f}")
print(f"Total Cost with Geographic Constraints: ${total_cost_with_constraints:,.2f}")
print(f"Total Cost with Manual Adjustments: ${total_cost_manual_adjustments:,.2f}")





--- Comparison of Remedies ---
Original Total Cost: $195,479.31
Total Cost with Geographic Constraints: $195,479.31
Total Cost with Manual Adjustments: $194,130.69


The original optimization achieved a total cost of **$195,479.31**, focusing solely on minimizing travel and salary costs without considering geographic feasibility.

Explicitly reassigning **Stafford_County to Warrenton** reduced the total cost to **$194,130.69**, showing that a targeted intervention can both enhance geographic feasibility and improve cost efficiency.